# Introducing heterochronicity in GNMs
In this example notebook, we use the gnm package to understand the concept of spatial heterochronicity. We will define a heterochronous gradient, and then perform a sweep over different parameter values to test whether introducing heterochronicity improves model fit.

In [1]:
import os
notebook_dir = os.getcwd() 

# able to run example script from within Jupyter notebook in package
import sys
loc = os.path.abspath(os.path.join(notebook_dir, '..', '..', 'src'))
sys.path.append(loc)

# other basic imports
import time
import torch
from gnm import *
from gnm import defaults, utils, evaluation, fitting, generative_rules, weight_criteria

C:\Users\fp02\AppData\Local\anaconda3\envs\gnm39\lib\site-packages\torch\_subclasses\functional_tensor.py:276: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: The specified module could not be found. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))
C:\Users\fp02\AppData\Local\anaconda3\envs\gnm39\lib\site-packages\wandb\sdk\launch\builder\build.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# device is the GPU if available, otherwise the CPU - GPU is much faster but ensure you have 
# GPU available in your PC/HPC

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# load the data (distance matrix and consensus network)
distance_matrix = defaults.get_distance_matrix(device=DEVICE)
binary_consensus_network = defaults.get_binary_network(device=DEVICE)

## Generating a heterochronicity gradient

After loading in some data just as in the Sweep Example, we need to generate a heterochronicity gradient. We can do this using the function generate_heterochronous_matrix() from the defaults sub-module.

First, let's see what arguments this function takes.

- coord: these are the coordinates of the nodes. They are needed to compute the distance between the origin point of heterochronicity and all the other points

In [4]:
coordinates = defaults.get_coordinates(device=DEVICE)

- reference_coord: this is the origin point. For now we will keep it fixed to the most caudal value, but it can also be let free to vary.

In [5]:
origin_point_idx = torch.argmin(coordinates[0,:])
origin_point = coordinates[origin_point_idx,:]
print(origin_point)

tensor([-38.6500,  -5.6800,  50.9400])


- sigma: the heterochronous gradients spreads from the origin point following a gaussian with some standard deviation sigma. Let's just set this to 15 for now.

In [6]:
sigma = 15

- num_nodes. This indicates the number of timesteps that the model will take. Usually, we add one connection at each timestep, so the number of timesteps equals to the number of conenctions we want to reach in the final synthetic model

In [7]:
num_connections = int( binary_consensus_network.sum().item() / 2 )
print(f"The binary consensus network contains {num_connections} connections.")

The binary consensus network contains 400 connections.


We'll run the models until they have the same number of connections as the real binary consensus network.

Feel free to explore the other arguments, but for now let's use the default values for those, and proceed generating the heterochronous matrix

In [8]:
from gnm.defaults.generate_heterochronous_matrix import generate_heterochronous_matrix


heterochronous_matrix = generate_heterochronous_matrix(
    coord = coordinates,
    reference_coord=origin_point,
    sigma=sigma,
    num_nodes=num_connections)


Now let's make a function to plot the heterochronous gradient.

> **Note:**  
> You may need to install `matplotlib` and `nilearn` via `pip` or `conda` to run the next plotting section!


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import plotting

def plot_heterochronous_evolution(
    hetero: torch.Tensor,
    coordinates: np.ndarray,
    n_steps: int = 7,
    node_cmap: str = 'viridis',
    node_size: int = 20,
    display_mode: str = 'x',
):
    """
    Visualise the heterochronous matrices at `n_steps` equally-spaced time points.
    Shows one colour-bar on the final panel only.
    """
    T_tot     = hetero.shape[2]
    time_idx  = np.linspace(0, T_tot-1, n_steps, dtype=int)

    fig, axes = plt.subplots(1, n_steps, figsize=(2.5*n_steps, 2.2))

    for k, t in enumerate(time_idx):
        ax  = axes[k]
        Ht  = hetero[:, :, t].cpu().numpy()
        vals = Ht.diagonal()                           # node intensities

        disp = plotting.plot_markers(
            vals,
            node_coords=coordinates,
            node_cmap=node_cmap,
            node_size=node_size,
            display_mode=display_mode,
            axes=ax,
            colorbar=(k == n_steps-1)                  # bar only on last subplot
        )
        if k != n_steps-1:             # drop colour-bar on all but last panel
            disp._colorbar.remove()

    plt.tight_layout()
    plt.show()


plot_heterochronous_evolution(heterochronous_matrix, coordinates, n_steps=7)


ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

## Defining our sweep

The next step is to define the parameters values we want to sweep over. Here, for simplicity, we'll keep $\eta$ and $\gamma$ fixed, while sweeping over a range of values for $\lambda$, which regulates the extent to which connections are formed following the heterochronous gradient.

We'll generate 10 networks using the model with that set of parameters. This means setting the number of simulations to 10. 

In [14]:
eta_value   = torch.tensor([-3.0])          
gamma_value = torch.tensor([0.2])
lambda_values = torch.linspace(0, 10, 5)


binary_sweep_parameters = fitting.BinarySweepParameters(
    eta = eta_value,
    gamma = gamma_value,
    lambdah = lambda_values,
    distance_relationship_type = ["powerlaw"],
    preferential_relationship_type = ["powerlaw"],
    heterochronicity_relationship_type = ["powerlaw"],
    generative_rule = [generative_rules.MatchingIndex()],
    num_iterations = [num_connections],
)


num_simulations = 10

sweep_config = fitting.SweepConfig(
    binary_sweep_parameters = binary_sweep_parameters,
    num_simulations = num_simulations,
    distance_matrix = [distance_matrix],   
    heterochronous_matrix = [heterochronous_matrix]
)

## Creating our evaluations

We want to evaluate how good the fit of our models is the real binary consensus network.
For our evaluation criteria, we'll use both topological metrics (the KS statistics across clustering coefficient, degree, betweenness centrality, and edge length distributions) and topographical metrics (correlations between real and synthetic networks for clustering coefficient, degree, and betweenness centrality).  

In [15]:
criteria = [ evaluation.ClusteringKS(), evaluation.DegreeKS(), evaluation.EdgeLengthKS(distance_matrix) ]
energy = evaluation.MaxCriteria( criteria )
binary_evaluations = [energy]

## Performing the sweep

In [16]:

experiments = fitting.perform_sweep(sweep_config=sweep_config, 
                                binary_evaluations=binary_evaluations, 
                                real_binary_matrices=binary_consensus_network,
                                save_model = False,
                                save_run_history = False,
)

Using device: None for GNM simulations


## Analysing the results

We can then check which value of $\lambda$ led to the lowest energy. If that's different from zero, we have reason to believe that introducing the heterochronous gradient was helpful in improving the model fit.

In [17]:
optimal_experiments, optimal_energies = fitting.optimise_evaluation(
    experiments=experiments,
    criterion=energy,
)

optimal_experiment = optimal_experiments[0]
optimal_energy = optimal_energies[0]

In [18]:
print(f"Optimal energy: {optimal_energy:0.3f}")
print(f"Optimal value of lambda: {optimal_experiment.run_config.binary_parameters.lambdah:0.2f}")


Optimal energy: 0.098
Optimal value of lambda: 10.00


We can see that $\lambda$ is greater than zero in the best model. We would have to perform appropriate statistical analyses to determine whether this value is significantly different from zero, but this is a good start!